In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import re
# from sklearn.decomposition import PCA
# import joblib
# from sklearn.metrics import f1_score
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import recall_score
# from sklearn.metrics import precision_score
# from sklearn.pipeline import Pipeline
# # TF IDF
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import cross_val_score, train_test_split # fnding max_features
# # from sklearn.ensemble import
# from sklearn.decomposition import PCA
# from sklearn.metrics import classification_report
# # import joblib
# import os

# # hyperparameter tuning
# from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier # to look availiable hyperparameters
# from pprint import pprint
# from sklearn.model_selection import RandomizedSearchCV #

# from hyperopt import fmin, tpe, hp

# import scipy.stats as stats
# import xgboost as xgb
# from xgboost import XGBClassifier
# from sklearn.model_selection import GridSearchCV

# import spacy
# import ast
# import sklearn
# import xgboost

# import matplotlib.pyplot as plt

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/Thesis/data/train/ner_features.csv')
test_tweet_data = pd.read_csv('/content/drive/MyDrive/Thesis/data/test/final_tweet_data.csv')

In [ ]:
# tweet data lebeld with falcon llm and gemini llm
tweet_data_f = pd.read_csv('/content/drive/MyDrive/Thesis/data/test/gold_tweet_test_falcon_7b.csv')
tweet_data_g = pd.read_csv('/content/drive/MyDrive/Thesis/data/test/gold_tweet_test_gemini.csv')

In [ ]:
print(tweet_data_f['label'].value_counts()) # falcon
print(tweet_data_g['label'].value_counts()) # gemini

label
0    176
1     59
Name: count, dtype: int64
label
No\n     128
Yes\n    107
Name: count, dtype: int64


In [ ]:
print(tweet_data_f.shape)
print(tweet_data_g.shape)

(235, 4)
(235, 4)


In [ ]:
tweet_data_g.head()

,Unnamed: 0,tweet_id,Text,label
0,0,1,dr. fauci must be charged for being a pandemic...,Yes\n
1,1,2,you guys remember when the government closed n...,Yes\n
2,2,3,the atlantic is trash and nobody believes that...,No\n
3,3,4,trump chose susan monarez to head the cdc. she...,Yes\n
4,4,5,the new york times finally admits the conspira...,Yes\n


In [ ]:
# modify some errors - drop columns, vaue changing
# tweet_data_f = tweet_data_f.drop('Unnamed: 0', axis = 1)
# tweet_data_g = tweet_data_g.drop('Unnamed: 0', axis = 1)

tweet_data_g['label'] = tweet_data_g['label'].apply(lambda x: re.sub('\n', ' ', x).strip())
tweet_data_g['label'] = tweet_data_g['label'].apply(lambda x: 1 if x == "Yes" else 0 )
tweet_data_g.head()

,Unnamed: 0,tweet_id,Text,label
0,0,1,dr. fauci must be charged for being a pandemic...,1
1,1,2,you guys remember when the government closed n...,1
2,2,3,the atlantic is trash and nobody believes that...,0
3,3,4,trump chose susan monarez to head the cdc. she...,1
4,4,5,the new york times finally admits the conspira...,1


In [ ]:
# save as final dataset, which will be use for testing the model
tweet_data_f.to_csv('/content/drive/MyDrive/Thesis/data/test/test_tweet_data_falcon_WL.csv', index = False)
tweet_data_g.to_csv('/content/drive/MyDrive/Thesis/data/test/test_tweet_data_gemini_WL.csv', index = False)

In [ ]:
test_tweet_data.rename(columns={'after_removing_nl;;;' : 'Text'}, inplace = True)

In [ ]:
test_tweet_data.iloc[1].isna()

,1
tweet_id,False
Text,False


In [ ]:
def column_readjusting(df):
  i = 1
  for index, row in test_tweet_data.iterrows():
    if pd.isna(row['Text']):
      text = str(row['tweet_id'])
      clean_text = re.sub(r'^(\d+),', '', text).strip()
      df.at[index, 'Text'] = clean_text

    df.at[index, 'tweet_id'] = int(i)
    i = i + 1


In [ ]:
column_readjusting(test_tweet_data)

In [ ]:
test_tweet_data.head()

,tweet_id,Text
0,1,dr. fauci must be charged for being a pandemic...
1,2,you guys remember when the government closed n...
2,3,the atlantic is trash and nobody believes that...
3,4,trump chose susan monarez to head the cdc. she...
4,5,the new york times finally admits the conspira...


In [ ]:
test_tweet_data.to_csv('/content/drive/MyDrive/Thesis/data/test/first_tweet_datas_1.csv', index = False)

In [ ]:
new_df = pd.read_csv('/content/drive/MyDrive/Thesis/data/test/first_tweet_datas_1.csv')

In [ ]:
new_df.tail()

,tweet_id,Text
95,96,"""what the hell """"leaky vaccine"""" usda conditio..."
96,97,"""randi weingarten is fighting to keep her sala..."
97,98,left vs right vs white vs black vs christian v...
98,99,"""she is not maga, and she is definitely not ma..."
99,100,dear @robertkennedyjr please let all 50 states...


In [ ]:
test_tweet_data['label'] = 0

In [ ]:
test_tweet_data.head()

,tweet_id,Text,label
0,1,dr. fauci must be charged for being a pandemic...,0
1,2,you guys remember when the government closed n...,0
2,3,the atlantic is trash and nobody believes that...,0
3,4,trump chose susan monarez to head the cdc. she...,0
4,5,the new york times finally admits the conspira...,0


In [ ]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM
import torch
from transformers import BitsAndBytesConfig

In [ ]:
# torch.manual_seed(0)
model_name =  "tiiuae/falcon-7b-instruct"

## to reducing the memory usages:
## https://medium.com/%40rakeshrajpurohit/model-quantization-with-hugging-face-transformers-and-bitsandbytes-integration-b4c9983e8996
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

model = AutoModelForCausalLM.from_pretrained(
          model_name,
          quantization_config=quantization_config,
          #load_in_4bit=True,
          device_map = "auto")

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

# pipe = pipeline(
#     "text-generation",
#     model = model,
#     tokenizer = tokenizer,
#     torch_dtype = torch.bfloat16,
#     device_map = "auto",
#     # max_new_tokens = 20,
#     # device=0 if torch.cuda.is_available() else -1,
# )

model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/17.7k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

FalconForCausalLM(
  (transformer): FalconModel(
    (word_embeddings): Embedding(65024, 4544)
    (h): ModuleList(
      (0-31): 32 x FalconDecoderLayer(
        (self_attention): FalconAttention(
          (query_key_value): Linear4bit(in_features=4544, out_features=4672, bias=False)
          (dense): Linear4bit(in_features=4544, out_features=4544, bias=False)
          (attention_dropout): Dropout(p=0.0, inplace=False)
          (rotary_emb): FalconRotaryEmbedding()
        )
        (mlp): FalconMLP(
          (dense_h_to_4h): Linear4bit(in_features=4544, out_features=18176, bias=False)
          (act): GELUActivation()
          (dense_4h_to_h): Linear4bit(in_features=18176, out_features=4544, bias=False)
        )
        (input_layernorm): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
      )
    )
    (ln_f): LayerNorm((4544,), eps=1e-05, elementwise_affine=True)
    (rotary_emb): FalconRotaryEmbedding()
  )
  (lm_head): Linear(in_features=4544, out_features=65024, bi

In [ ]:
# 26 11:00
# Check for GPU in Colab
# import torch

# if torch.cuda.is_available():
#     print("CUDA is available. GPU is detected.")
# else:
#     print("CUDA is not available. Please ensure you are using a GPU runtime in Colab.")
#     print("Go to Runtime -> Change runtime type -> Select GPU as Hardware accelerator")

# Install multi-platform bitsandbytes for Colab
# !pip install bitsandbytes-cuda120

- create function

In [ ]:
def ad_prompt_column(row):
    # action_type = ['Physical_Action', 'Mental_Action', 'State_Change', 'Create_or_Destruction', 'Communication', 'Movement', 'Emotion_or_Feeling' , 'Perception', 'Linking(State_of_being)Verb', 'Other']
    # noise_level = ['Relevant', 'Noisy']
    # Intent = ['Statement_of_Fact', 'Opinion_or_Belief', 'Argument_or_Justification', 'Action_or_Instruction', 'No _Clear_Intent']

    # sentiment = ['Positive', 'Negative', 'Other(Unclear/Noisy)']
    text = row['Text']
    # prompt_1 = f"Is the following text '{text}' Relevant or Noise. Tell me in one word"

    # prompt_2 = f"""Classify this {text} as 'Yes' (check-worthy) or 'No' (not check-worthy) based on:
    #           - If it contains a **verifiable factual claim**.
    #           - If it is **harmful** or **misleading**"""

    # this prompt used for classify the scrapped tweet text, to add their labels
    prompt_3 = (
    "Read the following statement carefully. "
    "Your task is to determine whether the statement contains claims or information that require fact-checking. "
    "- If the statement presents factual assertions, statistics, or claims that can be verified for accuracy, respond with 'Yes'. "
    "- If the statement is purely opinion-based, general knowledge, trivial, or lacks verifiable claims, respond with 'No'.\n\n"
    f"Statement: '{text}'\nResponse:"
    )

    # prompt_for_finding_action_verb = f"Is the following text '{text}' contains any action verb, Yes or No. Tell me in one word"

    # prompt_for_catagory_of_sentence = f"Can you catagorize this following text '{text}' into one of this categories: Social, Financial, Governmental, Political, Commercial, Constitutional and Environmental"
    # f"\n1. Relevant"
    # f"\n2. Noisy"
    # f"\n Generate Only the choosen option: "

            #         Intent (Choose One):
            #         - Statement of Fact
            #         - Opinion or Belief
            #         - Argument or Justification
            #         - Action or Instruction

            #         Sentiment (Choose One):
            #         - Positive
            #         - Negative
            #         - Neutral
            #         - Other (Unclear/Noisy)

            #         Provide the response in a structured format like:
            #         Action Type: [Category]
            #         Noise Level: [Category]
            #         Intent: [Category]
            #         Sentiment: [Category]
            # """
    #response =
    #print(response)
  ############################# generating lables for tweet data ()####################################################
    inputs = tokenizer(prompt_3, return_tensors = "pt").to(model.device)
    # print(inputs)

    outputs = model.generate(**inputs, max_new_tokens = 100, do_sample = False)
    # print('+'*30)
    # print('OUTPUT')
    # print(outputs)

    generate_result = tokenizer.decode(outputs[0], skip_special_tokens = True)
    # print("generate_result")
    # print(generate_result)

    print("+"*30)

    response = re.sub('\n', ' ', generate_result).strip().split(' ')[-1]

    if response == "'Yes'":
      row['label'] = 1
    else:
      row['label'] = 0
  #####################################################################################
    # inputs = tokenizer(prompt, return_tensors = "pt").to(model.device)
    # # print(inputs)

    # outputs = model.generate(**inputs, max_new_tokens = 100, do_sample = False)
    # print('+'*30)
    # print('OUTPUT')
    # print(outputs)

    # generate_result = tokenizer.decode(outputs[0], skip_special_tokens = True)
    # print("generate_result")
    # print(generate_result)

    # print("+"*30)

    # row['Noise_text'] = False
    # row['Relevant_text'] = False

    # if 'Noise' in generate_result.split():
    #     row['Noise_text'] = True
    # elif 'Relevant' in generate_result.split():
    #     row['Relevant_text'] = True
    # return row
  ##############################################################################################
    # start_indx = generate_result.find("Action Type: ")

    # print(generate_result[start_indx:])
    return row

In [ ]:
with torch.no_grad():
  test_tweet_data = test_tweet_data.apply(ad_prompt_column, axis = 1)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


++++++++++++++++++++++++++++++


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


++++++++++++++++++++++++++++++


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


...

++++++++++++++++++++++++++++++


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


++++++++++++++++++++++++++++++


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


++++++++++++++++++++++++++++++
++++++++++++++++++++++++++++++


In [ ]:
test_tweet_data['label'].value_counts()

,count
label,
0,176
1,59


In [ ]:
# save this data with lebel and without lebel
test_tweet_data.to_csv('/content/drive/MyDrive/Thesis/data/test/gold_tweet_test_falcon_7b.csv')
test_tweet_data.drop(['label'], axis = 1, inplace = True)
test_tweet_data.to_csv('/content/drive/MyDrive/Thesis/data/test/tweet_test_falcon_7b.csv')

In [ ]:
## using batch size to faster or efficient run
!nvidia-smi

Wed Feb 19 13:43:00 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   29C    P0             52W /  400W |    5005MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
with torch.no_grad():
  train_df = train_df.apply(ad_prompt_column, axis = 1)

Streaming output truncated to the last 5000 lines.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open

In [ ]:
train_df.to_csv('/content/drive/MyDrive/Thesis/data/train/ner_features_with_relevent_noise_prompt.csv', index = False)
# batch_size = 20
# for i in range(0, len(demo_train_df), batch_size):
#   batch_text = demo_train_df['Text'][i:i+batch_size].tolist()
#   print(batch_text)
#   results = ner_text(batch_text)
#   print(results)
#   demo_train_df.loc[i:i+batch_size-1, 'ner_result'] = results

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


['And so I know that this campaign has caused some questioning and worries on the part of many leaders across the globe.', "Now, let's balance the budget and protect Medicare, Medicaid, education and the environment.", "I'd like to mention one thing.", 'I must remind him the Democrats have controlled the Congress for the last twenty-two years and they wrote all the tax bills.', 'And to take a chance uh - now be - and not make every effort that we could make to provide for some control over these weapons, I think would be a great mistake.', "Well, what he is saying there in effect, we're going to have inflation.", "I'm proud of the fact that violent crime is down in the State of Texas.", 'You know, you may have seen your health care premiums go up.', "If we're $4 trillion down, we should have everything perfect, but we don't.", 'And I made some tough decisions.', 'I served for eight years in the House of Representatives and I served on the Intelligence Committee, specialized in looking 

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_tok

['}\n  }\n}\n\nTo extract the named entities from the text, you can use a regular expression to match the pattern of the entity name. Here\'s an example code snippet in Python:\n\n```python\nimport re\n\ntext = "The Golden State Warriors are an American professional basketball team based in San Francisco. Donald Trump is the president of Uganda. The team was valued at 4 billion dollars in 2022."\n\nnamed_entities = re.', '}\n  }\n}\n```\n\nIn this example, we have added a new category called "Money-related" and a new type called "Money" to the list of named entities. We have also added a new category called "Year or Date" and a new type called "Year" to the list of named entities.\n\nYou can modify the list of named entities and their categories as needed to fit your specific use case.', "}\n  }\n}\n\nHere's a modified version of your code that includes the additional types of named entities you mentioned:\n\n```\nimport nltk\nfrom nltk.tokenize import word_tokenize\nfrom nltk.stem imp

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.
Setting `pad_token_id` to `eos_tok

['}\n  }\n}\n```\n\nIn this example, we have a list of named entities that are categorized into different categories. The categories are defined in the `Categorization` class, which is a subclass of `NamedEntityRecognition`. The `Categorization` class is used to categorize the named entities in the text.\n\nIn the example, we have a list of named entities that are categorized into different categories. The categories are defined in the', '}\n  }\n}\n```\n\nIn this example, we have a list of named entities that are categorized by their type (e.g., organization, year, nationality, money, political party). We can then use this list to extract the relevant information from the text and perform further analysis.', '}\n  }\n}\n```\n\nIn this example, we have added a new category called "Money-related" and a new entity called "4 billion dollars". We have also added a new category called "Year or Date" and a new entity called "2022".\n\nYou can modify the code to add more categories and entiti

In [ ]:
def see_the_columns_values(id, df):
    print(f"Original text: {df[df['Sentence_id'] == id]['Text'].tolist()}")
    print(f"NER: {df[df['Sentence_id'] == id]['ner_result'].tolist()}")
    # print(f"names: {df[df['Sentence_id'] == id]['names'].tolist()}")
    # print(f"organizations: {df[df['Sentence_id'] == id]['organizations'].tolist()}")
    # print(f"locations: {new_df[new_df['Sentence_id'] == id]['locations'].tolist()}")
    # print(f"dates: {new_df[new_df['Sentence_id'] == id]['dates'].tolist()}")
    # print(f"verbs: {new_df[new_df['Sentence_id'] == id]['verbs'].tolist()}")
    # print(f"lemmatized verbs: {new_df[new_df['Sentence_id'] == id]['lemmatized_verb'].tolist()}")
    # print(f"action verbs: {new_df[new_df['Sentence_id'] == id]['action_verbs'].tolist()}")
    # print(f"filtered action verbs: {new_df[new_df['Sentence_id'] == id]['filtered_action_verbs'].tolist()}")
    # print(f"joined tokens: {new_df[new_df['Sentence_id'] == id]['joined_tokens'].tolist()}")
    # print(f"count verbs: {new_df[new_df['Sentence_id'] == id]['count_verb'].tolist()}")
    # print(f"count action verbs: {new_df[new_df['Sentence_id'] == id]['count_action_verb'].tolist()}")
    #  ###### 'names', 'organizations', 'locations','dates'
    # print(f"count action verbs: {new_df[new_df['Sentence_id'] == id]['names'].tolist()}")
    # print(f"count action verbs: {new_df[new_df['Sentence_id'] == id]['organizations'].tolist()}")
    # print(f"count action verbs: {new_df[new_df['Sentence_id'] == id]['locations'].tolist()}")
    # print(f"count action verbs: {new_df[new_df['Sentence_id'] == id]['dates'].tolist()}")

In [ ]:
see_the_columns_values(22058, demo_train_df)
# demo_train_df[demo_train_df['class_label'] == 'Yes']

Original text: ["If we're $4 trillion down, we should have everything perfect, but we don't."]
NER: ['}\n  }\n}\n```\n\nIn this example, we\'ve added a new category called "Money-related" and a new type called "Money" to the list of named entities. We\'ve also added a new category called "Year or Date" and a new type called "Year" to the list of named entities.\n\nNow, when you run the code, you\'ll see that the list of named entities has been updated to include the']


In [ ]:
train_df['informative_text_res'] = train_df['Text'].apply(classify_sentence)

In [ ]:
train_df.drop(['check_ner', 'informative_text_prompt'], axis = 1, inplace=True)

In [ ]:
# train_df['informative_text_prompt'] = train_df['Text'].apply(classify_sentence)

In [ ]:
# sequences = pipe(
#     prompt,
#     max_new_tokens = 100,
#     return_full_text = False,
# )

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


In [ ]:
# for seq in sequences:
#   print(f"Result: {seq['generated_text']}")
x = sequences[0]['generated_text']

In [ ]:
print(x)


1. ('Golden State Warriors', 'ORGANIZATION')
2. ('San Francisco', 'GPE')
3. ('Donald Trump', 'PER')
4. ('Uganda', 'GPE')
5. ('4 billion dollars', 'MONEY')
6. ('2022', 'YEAR')


Original text: []
NER: []


In [ ]:
text = "If we're $4 trillion down, we should have everything perfect, but we don't."
res = fun(text)

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


In [ ]:
print(res)

- "4" (number)
      - "trillion" (number)
      - "down" (adverb)
      - "perfect" (adjective)
      - "we" (pronoun)
      - "should" (conjunction)
      - "have" (verb)
      - "everything" (noun)
      - "perfect" (adjective)
      
